In [2]:
import os

In [3]:
def generate_command(filename, batch_size=None, model=None, experiment=None, debug=False):
    command = f'python {filename}'
    if batch_size is not None:
        command += f' --batch_size={batch_size}'
    if model is not None:
        command += f' --model={model}'
    if experiment is not None:
        command += f' --experiment={experiment}'
    if debug:
        command += ' --debug'

    return command

In [4]:
def generate_file(command, name):
    file_contents = f"""#!/bin/bash
#SBATCH -N 1
#SBATCH -c 4
#SBATCH -p ug-gpu-small
#SBATCH --gres=gpu
#SBATCH -t 00-18:00:00
#SBATCH --mem=28g
#SBATCH --qos=short
#SBATCH --job-name={name}
#SBATCH --mail-type=END,FAIL
#SBATCH --mail-user kqcr86@durham.ac.uk

eval "$('/home2/kqcr86/anaconda3/bin/conda' 'shell.bash' 'hook' 2> /dev/null)"
conda activate py310
{command}
"""
    return file_contents

In [5]:
def run_file(contents):
    with open('run.sbatch', 'w') as file:
        file.write(contents)

    return os.system('sbatch run.sbatch')

In [13]:
batch_sizes = {
    '41': 56,
    '42': 12,
    '43': 12,
    '44': 12,
    '45': 12
}

In [ ]:
for experiment in ['41', '42', '43', '44', '45']:
    command = generate_command('transf_fine_4.py', model='BioClinicalBert', experiment=experiment, batch_size=batch_sizes[experiment])
    file_contents = generate_file(command, name=f'bio_{experiment}',)
    run_file(file_contents)

In [19]:
for model in ['Bert', 'PubMedBert']:
    command = generate_command('transf_fine_4.py', model=model, experiment='41', batch_size=batch_sizes['41'])
    file_contents = generate_file(command, name=f'41_{model}')
    run_file(file_contents)

os.remove('run.sbatch')

Submitted batch job 250515
Submitted batch job 250516
Submitted batch job 250517
Submitted batch job 250518


In [20]:
for _ in range(250517, 250518+1):
    os.system(f'scancel {_}')